<a href="https://colab.research.google.com/github/s-grzhang/gwc-chatbot/blob/web-interface/gwc_bot_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT: Push to web-interface branch, not main branch.

In [1]:
!pip install transformers
!pip install datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
from datasets import Dataset

# Load the DataFrame from the CSV file
file_path = '/content/drive/My Drive/gwc_chatbot_data_revised.csv'
df = pd.read_csv(file_path)

# Format the dataset for Q&A, combining User Input (question) and Bot Response (answer)
df['input'] = df['User Input']
df['output'] = df['Bot Response']

# Create a Dataset from the DataFrame
dataset = Dataset.from_pandas(df[['input', 'output']])

# Display the dataset
print(dataset)


Dataset({
    features: ['input', 'output'],
    num_rows: 108
})


In [4]:
from transformers import AutoTokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if not already added
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset with both input (question) and output (answer)
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=128, return_attention_mask=True) #Try to make length longer
    outputs = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=128, return_attention_mask=True)
    # Shift outputs so they match the input for language modeling
    inputs["labels"] = outputs["input_ids"]
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/108 [00:00<?, ? examples/s]

In [5]:
# Split the dataset into train and test sets
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)


In [6]:
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Define the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4, #or 8
    per_device_eval_batch_size=4,
    num_train_epochs=10,  # You can adjust the number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.931000,3.176991
2,2.778700,2.690231
3,2.410600,2.478828
4,2.168600,2.357995
5,1.974400,2.296573
6,1.890800,2.273512
7,1.537800,2.258405
8,1.528000,2.239666
9,1.525000,2.237572
10,1.492800,2.232960


TrainOutput(global_step=220, training_loss=2.1373992096294057, metrics={'train_runtime': 48.3914, 'train_samples_per_second': 17.772, 'train_steps_per_second': 4.546, 'total_flos': 56177786880000.0, 'train_loss': 2.1373992096294057, 'epoch': 10.0})

In [7]:
import torch

# Ensure the model is on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_response(user_input):
    # Tokenize the input with attention_mask
    inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    # Generate a response with attention_mask
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,  # Pass attention_mask
        max_length=128,  # Set the maximum length of the generated response
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,  # Use EOS token as PAD token
        eos_token_id=tokenizer.eos_token_id   # Ensure EOS token is set
    )

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test the chatbot after fine-tuning
user_input = "What is the purpose of Girls Who Code?"
response = get_response(user_input)
print(response)


What is the purpose of Girls Who Code? Girls Who Code is to help girls learn computer science and computer science by participating in computer science clubs and other activities. Girls Who Code is a non-profit organization that does not provide any financial support to any other organizations. Girls Who Code does not provide any financial support to any other organizations. Girls Who Code does not provide any financial support to any other organizations. Girls Who Code does not provide any financial support to any other organizations. Girls Who Code does not provide any financial support to any other organizations. Girls Who Code does not provide any financial support to any other organizations. Girls Who Code does not


In [8]:
model.save_pretrained('/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit')
tokenizer.save_pretrained('/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit_tokenizer')


('/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit_tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit_tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit_tokenizer/vocab.json',
 '/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit_tokenizer/merges.txt',
 '/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit_tokenizer/added_tokens.json',
 '/content/drive/My Drive/gwc_chatbot_finetuned_not_overfit_tokenizer/tokenizer.json')

In [9]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.4 MB/s eta 0:00:00


In [10]:
def get_response(user_input, max_sentences=2):
    # Tokenize the user input and generate a response from GPT-2
    inputs = gpt2_tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=150)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to device
    outputs = gpt2_model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=128,
        num_return_sequences=1,
        top_p=0.9,
        no_repeat_ngram_size=3,
        pad_token_id=gpt2_tokenizer.eos_token_id,
        eos_token_id=gpt2_tokenizer.eos_token_id,
        do_sample=True
    )
    # Decode the GPT-2 response
    response = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Stop generation after max_sentences (e.g., 2 sentences)
    sentence_endings = ['.', '!', '?']  # Define sentence-ending punctuation
    sentence_count = 0
    final_response = ""

    # Iterate through characters and stop after two sentences
    for char in response:
        final_response += char
        if char in sentence_endings:
            sentence_count += 1
        if sentence_count == max_sentences:
            break

    return final_response

# Example usage
user_input = "What is the purpose of Girls Who Code?"
response = get_response(user_input, max_sentences=3)

print("Response with two sentences:")
print(response)


NameError: name 'gpt2_tokenizer' is not defined

In [ ]:
# Define paths to save the model and tokenizer
model_save_path = '/content/gwc_gpt2_model'
tokenizer_save_path = '/content/gwc_gpt2_tokenizer'

# Save the model and tokenizer
gpt2_model.save_pretrained(model_save_path)
gpt2_tokenizer.save_pretrained(tokenizer_save_path)


('/content/gwc_gpt2_tokenizer/tokenizer_config.json',
 '/content/gwc_gpt2_tokenizer/special_tokens_map.json',
 '/content/gwc_gpt2_tokenizer/vocab.json',
 '/content/gwc_gpt2_tokenizer/merges.txt',
 '/content/gwc_gpt2_tokenizer/added_tokens.json',
 '/content/gwc_gpt2_tokenizer/tokenizer.json')

In [ ]:
!zip -r /content/gwc_gpt2_model.zip /content/gwc_gpt2_model /content/gwc_gpt2_tokenizer


  adding: content/gwc_gpt2_model/ (stored 0%)
  adding: content/gwc_gpt2_model/generation_config.json (deflated 24%)
  adding: content/gwc_gpt2_model/model.safetensors (deflated 7%)
  adding: content/gwc_gpt2_model/config.json (deflated 50%)
  adding: content/gwc_gpt2_tokenizer/ (stored 0%)
  adding: content/gwc_gpt2_tokenizer/merges.txt (deflated 53%)
  adding: content/gwc_gpt2_tokenizer/vocab.json (deflated 59%)
  adding: content/gwc_gpt2_tokenizer/special_tokens_map.json (deflated 74%)
  adding: content/gwc_gpt2_tokenizer/tokenizer_config.json (deflated 56%)
  adding: content/gwc_gpt2_tokenizer/tokenizer.json (deflated 72%)


In [ ]:
from google.colab import files

# Download the zip file to your local machine
files.download('/content/gwc_gpt2_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>